# 1 Hosting static github website using kubernetes and helm

Set a unique name for your installation, to avoid disturbance. and configure git repo for the website you want to host

In [1]:
export RELEASE_NAME=jason; echo $RELEASE_NAME
# git repo you want to host as website
export GIT_REPO=https://github.com/mdn/beginner-html-site-styled.git; echo $GIT_REPO
# git branch for the static website
export GIT_BRANCH=master; echo $GIT_BRANCH


jason
https://github.com/mdn/beginner-html-site-styled.git
master


Install nginx according guide at https://artifacthub.io/packages/helm/bitnami/nginx

In [2]:
helm repo add bitnami https://charts.bitnami.com/bitnami

"bitnami" already exists with the same configuration, skipping


In [3]:
echo " 
replicaCount: 1
cloneStaticSiteFromGit:
  enabled: true
  repository: ${GIT_REPO}
  branch: ${GIT_BRANCH}
service:
  type: ClusterIP
ingress:
  enabled: true
  annotations:
    kubernetes.io/tls-acme: \"true\"
    cert-manager.io/cluster-issuer: issuer-nginx
    external-dns: \"true\"
    nginx.ingress.kubernetes.io/service-upstream: \"true\"
    nginx.ingress.kubernetes.io/upstream-vhost: ${RELEASE_NAME}-nginx.playground.svc.cluster.local
  ingressClassName: nginx
  hostname: ${RELEASE_NAME}-us.playground.findi.biz
  tls: true
" > bitnami.yaml ; cat bitnami.yaml

 
replicaCount: 1
cloneStaticSiteFromGit:
  enabled: true
  repository: https://github.com/mdn/beginner-html-site-styled.git
  branch: master
service:
  type: ClusterIP
ingress:
  enabled: true
  annotations:
    kubernetes.io/tls-acme: "true"
    cert-manager.io/cluster-issuer: issuer-nginx
    external-dns: "true"
    nginx.ingress.kubernetes.io/service-upstream: "true"
    nginx.ingress.kubernetes.io/upstream-vhost: jason-nginx.playground.svc.cluster.local
  ingressClassName: nginx
  hostname: jason-us.playground.findi.biz
  tls: true



In [4]:
helm upgrade --install -n playground ${RELEASE_NAME} bitnami/nginx --values bitnami.yaml

Release "jason" has been upgraded. Happy Helming!
NAME: jason
LAST DEPLOYED: Wed Sep 14 18:40:00 2022
NAMESPACE: playground
STATUS: deployed
REVISION: 4
TEST SUITE: None
NOTES:
CHART NAME: nginx
CHART VERSION: 13.2.4
APP VERSION: 1.23.1

** Please be patient while the chart is being deployed **
NGINX can be accessed through the following DNS name from within your cluster:

    jason-nginx.playground.svc.cluster.local (port 80)

To access NGINX from outside the cluster, follow the steps below:

1. Get the NGINX URL and associate its hostname to your cluster external IP:

   export CLUSTER_IP=$(minikube ip) # On Minikube. Use: `kubectl cluster-info` on others K8s clusters
   echo "NGINX URL: https://jason-us.playground.findi.biz"
   echo "$CLUSTER_IP  jason-us.playground.findi.biz" | sudo tee -a /etc/hosts


Check installation status

In [5]:
helm list -n playground

NAME 	NAMESPACE 	REVISION	UPDATED                                	STATUS  	CHART       	APP VERSION
jason	playground	4       	2022-09-14 18:40:00.278544546 +0000 UTC	deployed	nginx-13.2.4	1.23.1     


In [6]:
kubectl get pods -n playground

NAME                          READY   STATUS    RESTARTS   AGE
jason-nginx-fdbb78757-72l7r   3/3     Running   0          6m19s


In [7]:
kubectl get ingress -n playground

NAME          CLASS   HOSTS                           ADDRESS          PORTS     AGE
jason-nginx   nginx   jason-us.playground.findi.biz   34.168.186.113   80, 443   42m


Now access the url configured above for the website, it may take up to several minutes to let DNS update. 